\begin{center}
Gabe Morris
\end{center}

In [1]:
# Notebook Preamble
import sympy as sp
import matplotlib.pyplot as plt
from IPython.display import display

plt.style.use('maroon_ipynb.mplstyle')

\pagebreak
\tableofcontents
\pagebreak

\begin{center}
\begin{tabular}{c c c}
ME 4403 & Test 2 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 6-4
## Given
A steel rotating-beam test specimen has an ultimate strength of 1600 MPa.

## Find
Estimate the life of the specimen if it is tested at a completely reversed stress amplitude of 900 MPa.

## Solution
The first step is to find $S_e^\prime$.

In [2]:
sig_ar, S_ut = sp.S(900), sp.S(1600)

# Eq. 6-10
S_e_prime = sp.S(700)
S_e_prime  # MPa

700

The $S^\prime_e$ value will be used in place of $S_e$ from Figure 6-23 description. We can use the following relationships to determine $N$.

$$N=\left(\frac{\sigma_{ar}}{a}\right)^{1/b}$$
$$a=\frac{(fS_{ut})^2}{S_e}$$
$$b=-\frac{1}{3}\log\left(\frac{fS_{ut}}{Se}\right)$$

The value of $f$ is 0.77 from Figure 6-23 (estimated even though it is off the graph).

In [3]:
def log10(x_):
    return sp.log(x_)/sp.log(10)

f = sp.S('0.77')
a = (f*S_ut)**2/S_e_prime
b = -sp.Rational(1, 3)*log10(f*S_ut/S_e_prime)

display(sp.Eq(sp.Symbol('a'), a.n()),
        sp.Eq(sp.Symbol('b'), b.n()))

N = ((sig_ar/a)**(1/b)).n()
N  # cycles

Eq(a, 2168.32)

Eq(b, -0.0818375559380499)

46379.6905856764

The life of the specimen is 46400 cycles.

\pagebreak

\begin{center}
\begin{tabular}{ccc}
ME 4403 & Test 2 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 6-11
## Given

\begin{center}
\includegraphics{images/fig1}
\end{center}

A pin in a knuckle joint is shown in part (a) of the figure above. The joint is subject to a repeatedly applied and released load of 6000 N in tension. Assume the loading on the pin is modeled as concentrated forces as shown in part (b) of the figure. The shaft is made from AISI 1018 hot-rolled steel that has been machined to its final diameter.

In [4]:
# Table A-20
Sut, Sy = sp.S(400), sp.S(220)  # MPa

## Find
Based on a stress element on the outer surface at the cross-section A, determine a suitable diameter of the pin, rounded up to the next mm increment, to provide at least a factor of safety of 1.5 for both infinite fatigue life and for yielding.

## Solution
### Yielding

\begin{center}
\includegraphics{images/fig2}
\end{center}

The equation for the maximum stress may be found using the bending moment diagram above.

In [5]:
d = sp.Symbol('d', real=True, positive=True)
M = 45_000  # N*mm^2
c = d/2
I = sp.pi/64*d**4
sig_max = (M*c/I).n()
sig_max

458366.236104659/d**3

For yielding,

$$n=\frac{S_y}{\sigma_{max}}$$

In [6]:
eq1 = sp.Eq(sp.S('1.5'), Sy/sig_max)
display(eq1)
d_ = sp.solve(eq1)[0]
display(sp.Eq(d, d_))

Eq(1.5, 0.000479965544298441*d**3)

Eq(d, 14.6204385310094)

So for yielding, a suitable diameter would be $d=15\,mm$.

### Infinite Life
The Goodman method will be used to determine the diameter that results in an infinite life with a factor of safety of 1.5. The relationship for this is,

$$\frac{1}{n_f}=\frac{\sigma_a}{S_e}+\frac{\sigma_m}{S_{ut}}$$

The load is not reversible, but is fluctuating on and off, so the mean stress is positive.

In [7]:
# Getting Se
# Equations come from the road map starting on page 359

S_e_prime = Sut/2

a, b = sp.S('3.04'), sp.S('-0.217')  # Table 6-2 (Machined)
k_a = (a*Sut**b).n()
k_b = sp.S('1.24')*d**sp.S('-0.107')

Se = k_a*k_b*S_e_prime
Se

205.436802219618/d**0.107

In [8]:
# Applying the Goodman relationship
sig_a = sig_m = sig_max/2  # minimum stress is 0
eq1 = sp.Eq(sig_a/Se + sig_m/Sut, 1/sp.S('1.5'))
display(eq1)
d_inf = sp.nsolve(eq1, d, 10)  # Numerical solver
display(sp.Eq(d, d_inf))

Eq(1115.58939574676/d**2.893 + 572.957795130823/d**3, 0.666666666666667)

Eq(d, 14.5624553895619)

The above equation is complex and the diameter can only be solved using the numerical solver in `sympy`. The results show that the $15\,mm$ diameter should be used for both yielding and infinite life.

\pagebreak

\begin{center}
\begin{tabular}{ccc}
ME 4403 & Test  2 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 3-61
## Given
A machine part will be cycled at $\pm350\,MPa$ for $5\cdot10^3$ cycles. Then loading will be changed to $\pm260\,MPa$ for $5\cdot10^4$ cycles. Finally, the load will be changed to $\pm225\,MPa$.

$$S_{ut}=350\,MPA$$
$$f=0.9$$
$$S_e=210\,MPa$$

In [9]:
Sut, f, Se = sp.S(530), sp.S('0.9'), sp.S(210)

## Find
How many cycles of operation can be expected at this stress level using Miner's method?

## Solution
Minor's method is,

$$\frac{n_1}{N_1}+\frac{n_2}{N_2}+\frac{n_3}{N_3}=1$$

In [10]:
def get_N(s_max, s_min, f_value, Sut_value, Se_value):
    s_m = (s_max + s_min)/2
    s_a = (s_max - s_min)/2
    s_ar = s_a/(1 - s_m/Sut_value)
    a_ = (f_value*Sut_value)**2/Se_value
    b_ = -sp.Rational(1, 3)*log10(f_value*Sut_value/Se_value)
    N_ = (s_ar/a_)**(1/b_)
    return N_.n()

n1, n2 = sp.S(5e3), sp.S(5e4)
N1 = get_N(sp.S(350), -sp.S(350), f, Sut, Se)
N2 = get_N(sp.S(260), -sp.S(260), f, Sut, Se)
N3 = get_N(sp.S(225), -sp.S(225), f, Sut, Se)

n3 = sp.Symbol('n_3')
eq1 = sp.Eq(n1/N1 + n2/N2 + n3/N3, 1)
display(eq1)
n3_ = sp.solve(eq1)[0]
display(sp.Eq(n3, n3_))

Eq(1.78766904825898e-6*n_3 + 0.670863205354351, 1)

Eq(n_3, 184115.060316224)

\pagebreak

\begin{center}
\begin{tabular}{ccc}
ME 4403 & Test 2 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 8-12
## Given
An $M14\times2$ hex head bolt with a nut is used to clamp together two 15 mm steel plates. There is a 14R metric plain washer under the nut.

## Find
a. Determine a suitable length for the bolt, rounded up to the nearest 5 mm.
b. Determine the bolt stiffness.
c. Determine the stiffness of the members.

## Solution
### Part A
The bolt dimensions are found in Table A-31.

In [11]:
l, H, t = sp.S(30), sp.S('12.8'), sp.S('3.5')
L = l + H + t
L  # mm

46.3000000000000

The bolt length is $50\,mm$.

### Part B
The fastener stiffness along with more relationships come from Table 8-7.

$$k_b=\frac{A_dA_tE}{A_dl_t+A_tl_d}$$

In [12]:
d = sp.S(14)  # mm
E = sp.S('207')  # Pa
L = 50  # mm

Ad = sp.pi*d**2/4
At = sp.S(115)
Lt = 2*d + 6
ld = L - Lt
lt = l + t - ld
kb = Ad*At*E/(Ad*lt + At*ld)
kb.n()  # MN/m

808.240665447356

### Part C

In [13]:
# Eq. 8-22
x = sp.S('0.5774')
km = (x*sp.pi*E*d)/(2*sp.log(5*(x*(l + t) + sp.S('0.5')*d)/(x*(l + t) + sp.S('2.5')*d)))
km.n()  # MN/m

2968.88532206290